In [ ]:
import os
import PIL
import pathlib
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from sklearn.metrics import accuracy_score

# Telegram Bot

In [ ]:
personal_token = "5659698721:AAG7KgUeQiMP42oIGOvQtPouTLnCwbPNhnA"

In [ ]:
class botCallback(tf.keras.callbacks.Callback):
  def __init__(self,personal_token):
    self.personal_token = personal_token
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.chat_id = self.response['result'][0]['message']['chat']['id']
    self.last_message_id = self.response['result'][-1]["message"]["message_id"]

  def send_message(self,message):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/sendMessage?'+\
                    'chat_id='+str(self.chat_id)+\
                    '&parse_mode=Markdown'+\
                    '&text='+message
    self.response = requests.get(self.ping_url)

  def check_stop(self):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.msg = self.response['result'][-1]["message"]["text"]
    self.message_id = self.response['result'][-1]["message"]["message_id"]
    if self.message_id != self.last_message_id and self.msg == 'stop':
      self.model.stop_training = True
      self.send_message("Training Stopped!")
    else:
      self.model.stop_training = None
    self.last_message_id = self.message_id

  def on_train_begin(self, logs=None):
    self.send_message("Training Begins!")

  def on_epoch_end(self, epoch, epoch_logs):
    message = ' epoch {}\n Training Accuracy : {:7.3f}\n Training Loss : {:7.3f}\n Validation Accuracy : {:7.3f}\n Validation Loss : {:7.3f}\n'.format(epoch+1,epoch_logs['acc'],epoch_logs['loss'],
                                                                                                                                                 epoch_logs['val_acc'],epoch_logs['val_loss'])

    self.send_message(message)
    self.check_stop()

  def on_train_end(self, logs=None):
    self.send_message("Training Completed!")

  def on_test_end(self,logs=None):
    self.send_message('Test results: Accuracy of {:7.3f}% '.format(logs['acc']*100))

In [ ]:
bot_callback = botCallback(personal_token)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = pathlib.Path('/content/drive/MyDrive/DSSS HW9 DataSet/Images_Real')

def data_loader():
  global train_ds
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32)

  global val_ds
  val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=0.1,
      subset="validation",
      seed=123,
      image_size=(256, 256),
      batch_size=32)

  global test_ds
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=0.1,
      subset="validation",
      seed=123,
      image_size=(256, 256),
      batch_size=32)

In [ ]:
data_loader()

Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 300 files for validation.
Found 3000 files belonging to 3 classes.
Using 300 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Cat', 'Dog', 'Siamese']


# **Training Model:**

In [ ]:
resnet_model = Sequential()

pretrained_model= ResNet50(include_top=False,
                   input_shape=(256,256,3),
                   pooling='max',
                   classes=3,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dropout(0.2)) # This is the layer that i added
resnet_model.add(Dense(3, activation='softmax'))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 64)               131136    
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 128)              8320      
 pper)                                                           
                                                                 
 module_wrapper_3 (ModuleWra  (None, 256)              33024     
 pper)                                                  

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks = bot_callback)

Epoch 1/10
75/75 [==============================] - 579s 7s/step - loss: 0.3171 - acc: 0.9004 - val_loss: 0.0862 - val_acc: 0.9767
Epoch 2/10
75/75 [==============================] - 15s 189ms/step - loss: 0.0444 - acc: 0.9829 - val_loss: 0.0731 - val_acc: 0.9800
Epoch 3/10
75/75 [==============================] - 14s 185ms/step - loss: 0.0232 - acc: 0.9929 - val_loss: 0.0682 - val_acc: 0.9800
Epoch 4/10
75/75 [==============================] - 14s 183ms/step - loss: 0.0082 - acc: 0.9958 - val_loss: 0.0882 - val_acc: 0.9800
Epoch 5/10
75/75 [==============================] - 14s 184ms/step - loss: 0.0269 - acc: 0.9925 - val_loss: 0.1862 - val_acc: 0.9433
Epoch 6/10
75/75 [==============================] - 14s 188ms/step - loss: 0.0384 - acc: 0.9879 - val_loss: 0.0742 - val_acc: 0.9833
Epoch 7/10
75/75 [==============================] - 14s 183ms/step - loss: 0.0356 - acc: 0.9904 - val_loss: 0.0574 - val_acc: 0.9900
Epoch 8/10
75/75 [==============================] - 15s 190ms/step - lo

## Evaluating the Model

# Making predictions

In [ ]:
pred = resnet_model.predict(test_ds)
print(pred)

In [ ]:
resnet_model.evaluate(test_ds, callbacks = bot_callback)

10/10 [==============================] - 3s 218ms/step - loss: 0.0822 - acc: 0.9800


[0.08220073580741882, 0.9800000190734863]